In [ ]:
%reload_ext autoreload
%autoreload 2

from importlib import reload
import os
import json
import logging
import datetime

import requests
import tqdm.notebook as tqdm

import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

import zcode.inout as zio
import zcode.math as zmath
import zcode.plot as zplot

import kalepy as kale

import weathervane as wv

In [ ]:
PATH_STATION_DATA = "/Users/lzkelley/Programs/weathervane/data/stations"

FNAME_RAW_HISTORY = "/Users/lzkelley/Programs/weathervane/data/raw/isd-history.csv"
FNAME_STATIONS = "/Users/lzkelley/Programs/weathervane/data/us-stations_filtered.csv"
FNAME_INVENTORY = "/Users/lzkelley/Programs/weathervane/data/raw/isd-inventory.csv"

In [ ]:
def sci_not(val, **kwargs):
    kwargs.setdefault('man', 1)
    kwargs.setdefault('dollar', False)
    kwargs.setdefault('sign', True)
    return zplot.scientific_notation(val, **kwargs)

# Select a subset of weather stations, save to file

In [ ]:
# df = pd.read_csv(FNAME_RAW_HISTORY)
# print(df.keys(), "\n", df.size)

# country = df['CTRY']
# beg = df['BEGIN']
# end = df['END']
# state = df['STATE']
# idx = (country == 'US') & (end > 20200000) & (beg < 20000000) & (~state.isnull())
# print(np.count_nonzero(idx))

# stations = df.loc[idx]
# # df_filt = df_filt.sort_values('STATE')

# stations.to_csv(FNAME_STATIONS, index_label='global index')
# print(f"Saved to {FNAME_STATIONS}, size {zio.get_file_size(FNAME_STATIONS)}")

In [ ]:
# stations = wv.load_stations_list()
reload(wv)
reload(wv.main)
reload(wv)
stations = wv.Stations()

In [ ]:
logging.getLogger().setLevel(0)
stations.nearest('santa cruz')

# Load 'Inventory'

In [ ]:
# inventory = pd.read_csv(FNAME_INVENTORY)
# print(f"Loaded {inventory.size} entries from '{FNAME_INVENTORY}'\n\tkeys={inventory.keys()}")

# Download data for all city-matching stations

In [ ]:
logging.getLogger().setLevel(30)

In [ ]:
stat

In [ ]:
cities = wv.Cities()
dists = np.zeros(cities.size)
indices = np.zeros_like(dists, dtype=int)
for ii, (city, state) in enumerate(zio.tqdm(cities, total=cities.size, desc='cities', leave=True)):
    name = city + ", " + state
    stat, idx, dd = stations.nearest(name, warn_above=None)
    dists[ii] = dd
    indices[ii] = idx
    wv.download_station(stat, warn_404=False)
    

# Download data for target station

In [ ]:
# idx = stations['STATION NAME'].str.contains(r'san', case=False)
# print(idx)
# stations.loc[19688]

idx = stations['STATION NAME'].str.contains(r'chicago', case=False)
stations.loc[idx]

In [ ]:
# station_call = "KSFO"
station_call = "KORD"
stat = wv.get_station(call=station_call)
wv.download_station(stat)
data = wv.load_station_data(stat)

# Examine Data

In [ ]:
def temp_conv(df_vals):
    # df_vals = df['TMP']
    # Convert from object array to 1D np.array of 7-char strings
    temp = df_vals.to_numpy().astype('S7')
    # Convert to 2D array of characters
    temp = temp.view(dtype='S1').reshape(temp.size, len(temp[0]))
    # Get quality code
    qq = temp[:, 6]   # .view(np.uint8) - ord('0')
    # Get sign
    ss = temp[:, 0]
    # convert to integers, combine, convert to float temperature    
    temp = temp[:, 1:5].view(np.uint8) - ord('0')
    aa = np.array([1000, 100, 10, 1])
    temp = np.sum(temp * aa[np.newaxis, :], axis=1) / 10.0
    # Add back sign
    temp = temp * (1 + -2 * (np.array(ss) == b'-'))
    
    return temp, qq


def station_to_grid(data, tzone='US/Pacific'):
    raw_temp, qq = temp_conv(data['TMP'])
    sel = (qq == b'1') | (qq == b'5')  # | (qq != )
    logging.info(f"valid temperatures = {zmath.frac_str(sel)}")

    # ---- Make sure that data starts and ends at year boundaries ----
    time = data['DATE'].loc[sel]
    beg_year = int(time.iloc[0].year)
    last = time.size - 1
    end_year = time.iloc[-1].year
    logging.info(f"beg date: {time.iloc[0]}, year: {beg_year}")
    logging.info(f"end date: {time.iloc[last]}, year: {end_year}")

    beg = pd.Timestamp(f'{beg_year}-01-01T01:30:00', tz=tzone)
    if time.iloc[0] >= beg:
        beg = pd.Timestamp(f'{beg_year+1}-01-01T00:00:00', tz=tzone)
        sel = sel & (data['DATE'] >= beg)
    
    end = pd.Timestamp(f'{end_year}-12-31T22:30:00', tz=tzone)
    if time.iloc[last] <= end:
        end = pd.Timestamp(f'{end_year}-01-01T00:00:00', tz=tzone)
        sel = sel & (data['DATE'] < end)

    logging.info(f"valid temps and dates = {zmath.frac_str(sel)}")
    data = data.loc[sel]
    raw_temp = raw_temp[sel]
    time = data['DATE']

    # Make sure it worked
    last = time.size - 1
    beg_year = time.iloc[0].year
    if (time.iloc[0].day != 1) or (time.iloc[0].month != 1):
        raise ValueError(f"Dataset does not start on Jan. 1st!!  ({time[0]})")

    # Convert to proper timezone
    time = time.dt.tz_convert(tzone)
    
    # Determine the number of complete years covered
    nyrs = (time.iloc[last] - time.iloc[0]).total_seconds() / (3600*24*365)
    nyrs = int(np.floor(nyrs))
    logging.info(f"{beg_year=}, {time.iloc[0]}, {time.iloc[last]}, {nyrs=}")
    
    # ---- Find mapping of entries to target grid points ----
    
    # Initialize storage array, and create grid of hours after 'Jan. 1st' each year
    index = -1 * np.ones((nyrs, 365, 24), dtype=int)
    global_index = -1 * np.ones((nyrs, 365, 24), dtype=int)

    dt_grid = np.arange(365*24).astype(float)

    # Iterate over years, find the indices of entries matching each offset for that year
    years = -1 * np.ones(nyrs, dtype=int)
    for yy in tqdm.trange(nyrs):
        year = beg_year + yy
        beg = pd.Timestamp(f'{year}-01-01', tz=tzone)
        end = pd.Timestamp(f'{year}-12-31T23:59:59', tz=tzone)

        # Find the entries for this year
        idx = (time >= beg) & (time <= end)
        # Determine the offset (index) for the first value
        offset = np.argmax(idx)
        num_year = np.count_nonzero(idx)
        epd = num_year / 365.0
        msg = f"{year=}, {offset=}, #/day={epd:.2f}"
        logging.debug(msg)
        if epd < 1.0:
            logging.error(msg)
            err = f"Entries per day is {epd:.2e} < 1!!"
            logging.error(err)
            # raise ValueError(err)
            continue
            
        years[yy] = year
        
        # Find the time of each entry relative to 'Jan 1st', in hours
        dt = (time[idx] - beg).dt.total_seconds() / 3600.0
        # Find the nearest entry matching each target grid-point
        sel = zmath.argnearest(dt, dt_grid, assume_sorted=True)
        # Store values
        index[yy, ...].flat = offset + sel
        
    # ---- Select valid years ----
    valid = (years > 0)
    if not np.all(valid):
        logging.warning(f"Valid years: {zmath.frac_str(valid)}")
        years = years[valid]
        index = index[valid, ...]
        global_index = global_index[valid, ...]        
        
    # ---- Interpolate ----
    
    temp = np.zeros_like(index, dtype=float)
    idx = index.flatten()
    temp.flat = raw_temp[idx]
    global_index.flat = data.iloc[idx].index.to_numpy()
    
    return years, temp, global_index

logging.getLogger().setLevel(20)
years, temp, gidx = station_to_grid(data);
print(temp.shape, zmath.stats_str(temp))
# noise = np.clip(np.random.normal(scale=0.5, size=temp.shape), -1.5, +1.5)
# temp = sp.constants.convert_temperature(temp, 'Celsius', 'Fahrenheit') + noise
temp = sp.constants.convert_temperature(temp, 'Celsius', 'Fahrenheit')

In [ ]:
tave = np.mean(temp, axis=0)
levels = zmath.spacing(tave/10, scale='lin', integers=True) * 10
print(levels)

fig, ax = plt.subplots(figsize=[16, 6])
pcm = ax.pcolormesh(tave.T)
cbar = plt.colorbar(pcm, label='Temperature [F]')

ax.axhline(12, color='0.5', ls='--')

edges = [np.arange(365), np.arange(24)]
*_, cc = kale.plot.draw_contour2d(
    ax, edges, tave, cmap=pcm.cmap.reversed(),
    levels=levels, smooth=3, upsample=2, pad=2, cbar=cbar
)

ax.set(xlim=[0, 365], xlabel='Day of Year', ylim=[0, 24], ylabel='Hour of Day')
plt.show()

In [ ]:
# levels = [50, 60, 65]
levels = None

nyr, nday, nhr = temp.shape
td_ave = np.mean(temp, axis=-1).T
edges = [np.arange(nday), years]

fig, ax = plt.subplots(figsize=[16, 6])
pcm = ax.pcolormesh(*edges, td_ave.T, shading='auto')
cbar = plt.colorbar(pcm, label='Ave Temperature [F]')

*_, cc = kale.plot.draw_contour2d(
    ax, edges, td_ave,
    levels=levels, smooth=3, upsample=2, pad=2, cbar=cbar
)

ax.set(xlim=[0, nday], xlabel='Day of Year', ylim=zmath.minmax(years), ylabel='Year')
plt.show()

In [ ]:
fig, axes = zplot.figax(ncols=2, scale='lin', sharey=True)
confs = [50, 90]

shp = temp.shape

def plot_ave_axis(ax, temp, axis):
    test = np.moveaxis(temp, axis, 0).reshape(shp[axis], -1)
    tave = test.mean(axis=-1)
    hh, = ax.plot(tave)

    for pp in confs:
        conf = np.percentile(test, [50-pp/2, 50+pp/2], axis=-1)
        ax.fill_between(np.arange(tave.shape[0]), *conf, alpha=0.2, color=hh.get_color())

    return

plot_ave_axis(axes[0], temp, 1)
plot_ave_axis(axes[1], temp, 2)

plt.show()


In [ ]:
confs = [20, 40]

plt.figure(figsize=[12, 4])
plt.grid(alpha=0.2)

test = temp.reshape(np.shape(temp)[0], -1)
tyr_ave = np.mean(test, axis=-1)
tyr_med = np.median(test, axis=-1)
# xx = np.arange(test.shape[0])
xx = years

ave_coeff, ave_fit = zmath.numeric.regress(xx, tyr_ave)

hh, = plt.plot(xx, tyr_ave, ls=':', label='ave')
col = hh.get_color()
plt.plot(xx, tyr_med, color=col, ls='--', label='med')
plt.plot(xx, ave_fit, color=col, ls='-', label='ave fit')
plt.title(fr'${sci_not(ave_coeff[0])} \; \mathrm{{deg/yr}}$')

for pp in confs:
    conf = np.percentile(test, [50-pp/2, 50+pp/2], axis=-1)
    plt.fill_between(xx, *conf, alpha=0.2, label=fr'${pp}\%$', color=col)

plt.legend()
plt.show()


In [ ]:
temp.shape
xx = years[:, np.newaxis, np.newaxis] * np.ones_like(temp)

coeff, zz = zmath.numeric.regress(xx, temp)
slope = coeff[0]

levels = None
# levels = zmath.spacing(tave/10, scale='lin', integers=True) * 10
# print(levels)
# levels = [-0.01, 0.01]

smap = zplot.smap(slope, cmap='RdBu_r', midpoint=0.0)
cmap = smap.cmap
# cmap = 'RdBu_r'

fig, ax = plt.subplots(figsize=[16, 6])
pcm = ax.pcolormesh(slope.T, cmap=smap.cmap, norm=smap.norm)
cbar = plt.colorbar(pcm, label=r'$\Delta$ Temperature [F/yr]')

ax.axhline(12, color='0.5', ls='--')

edges = [np.arange(365), np.arange(24)]
*_, cc = kale.plot.draw_contour2d(
    ax, edges, slope, cmap=pcm.cmap.reversed(),
    levels=levels, smooth=3, upsample=2, pad=2, cbar=cbar
)

ax.set(xlim=[0, 365], xlabel='Day of Year', ylim=[0, 24], ylabel='Hour of Day')
plt.show()

In [ ]:
print(temp.shape)
tave = np.mean(temp, axis=-1)
xx = years[:, np.newaxis] * np.ones_like(tave)

coeff, zz = zmath.numeric.regress(xx, tave)
slope = coeff[0]
print(slope.shape)

plt.plot(slope)


In [ ]:
fig, axes = zplot.figax(ncols=2, scale='lin', sharey=True)
confs = [50, 90]

shp = temp.shape

def plot_ave_axis(ax, years, temp, axis, smap=None):
    other = 2 if (axis == 1) else 1
    xlab = 'day of year' if (axis == 1) else 'hour of day'
    tave = np.mean(temp, axis=other)

    xx = years[:, np.newaxis] * np.ones_like(tave)
    slope, zz = zmath.numeric.regress(xx, tave)
    slope = slope[0]
    if smap is None:
        smap = zplot.smap(slope, scale='lin', cmap='RdBu_r', midpoint=0.0)
    colors = smap.to_rgba(slope)
    
    test = np.moveaxis(temp, axis, 0).reshape(shp[axis], -1)
    tave = test.mean(axis=-1)
    xx = np.arange(tave.size)

    hh, = ax.plot(xx, tave)
    ax.scatter(xx, tave, color=colors)

    for pp in confs:
        conf = np.percentile(test, [50-pp/2, 50+pp/2], axis=-1)
        ax.fill_between(xx, *conf, alpha=0.2, color=hh.get_color())

    plt.colorbar(smap, ax=ax, orientation='horizontal', label=r'$\Delta T$ [deg/yr]')
    ax.set(xlabel=xlab, ylabel='Temperature [F]')
    
    return smap

plot_ave_axis(axes[0], years, temp, 1)
plot_ave_axis(axes[1], years, temp, 2)

plt.show()


In [ ]:
hrs = df['DATE'].dt.hour + df['DATE'].dt.minute/60 + df['DATE'].dt.second/3600
hrs = hrs.to_numpy()

days = df['DATE'].dt.dayofyear.to_numpy()

In [ ]:
dt = df['DATE'] - df['DATE'][0]
dt = np.diff(dt.dt.total_seconds())

In [ ]:
plt.figure(figsize=[18, 6])
kale.dist1d(dt[::100]/3600, probability=True, density=True);

In [ ]:
plt.hist(hrs, bins=24*10);

In [ ]:
kk = 'RH1'
df[kk][~df[kk].isnull()]

In [ ]:
for kk in df.keys():
    print(kk)

# List of US cities by population

In [ ]:
stations

In [ ]:
reload(wv)
reload(wv.main)
reload(wv)
cities = wv.Cities()

In [ ]:
cities._data

In [ ]:
FNAME_CITIES = "/Users/lzkelley/Programs/weathervane/data/raw/SUB-IP-EST2019-ANNRNK.xlsx"
names = ["rank", "geographic area", "2010 census", "2010 estimates base",]
names = names + [str(yy) for yy in range(2010, 2020)]
# print(names)
header = list(range(3))
footer = list(range(792, 800))
df = pd.read_excel(FNAME_CITIES, names=names, skiprows=header + footer)
print(df.keys(), "\n", df.size)
# df['geographic area']
df[:10]